<a href="https://colab.research.google.com/github/MSjoia/Drugs-Identification/blob/main/Codes_Match_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drugs Identification Application:**
###**Purpose:**
Standardize input medicine/item names to identify a medicine/item from the database.
###**Scope:**
This application aims to match Internal Codes given the Supplier Codes and/or PIP Codes and/or Supplier Descriptions.

It comprises of 2 independent programs:
<br>i) Internal Codes Match
<br>ii) NHSBSA Descriptions Match
<br>iii) Future Extension: AI verification for the matched results
###<br>**How does it work?**
**i) Internal Codes Match:**
1.   If given, it finds and matches supplier codes with the (if) existing Supplier Template in Templates Google Drive Folder.

| Parameter           | Value             |
|---------------------|-------------------|
| Input:              | Supplier Code     |
| Match With:         | Supplier Template |
| Output:             | Internal Code     |
| Match Type:         | Exact Match       |
| Match Type Code:    | Supplier Code     |
| Score:              | N/A (Blank)       |

2.   For the remaining lines, it matches PIP Codes with all of the templates and PricelistExport file database. PIP Codes are assumed to be constant across different templates.

| Parameter           | Value             |
|---------------------|-------------------|
| Input:              | PIP Code          |
| Match With:         | PIP Codes of All Templates & PricelistExport|
| Output:             | Internal Code     |
| Match Type:         | Exact Match       |
| Match Type Code:    | PIP - Templates/PricelistExport|
| Score:              | N/A (Blank)       |

3.   For unmatched lines, it matches input descriptions with all of the templates descriptions and PricelistExport database.

| Parameter           | Value             |
|---------------------|-------------------|
| Input:              | Description       |
| Match With:         | Descriptions of All Templates & PricelistExport|
| Output:             | Internal Code     |
| Match Type:         | Exact Match       |
| Match Type Code:    | Item - Templates/PricelistExport|
| Score:              | N/A (Blank)       |

4.   As a last resort it uses input descriptions and matches with standard database description as a fuzzy match with a confidence threshold of 70%. Best match with highest percent resemblance is chosen, with minimum resemblence being 70%. To increase accuracy item name is first used to filter all the relevant results with 90% threshold. This leaves a very slim chance of mismatch only in case of anomalies like Omeprazole and Esomeprazole which are different items but highly alike.

| Parameter           | Value             |
|---------------------|-------------------|
| Input:              | Description       |
| Match With:         | PricelistExport Database|
| Output:             | Internal Code     |
| Match Type:         | Approximate Match |
| Match Type Code:    | Percentage        |
| Score:              | Number (How much % match)|


<br>**ii) NHSBSA Description Match:**
1.   An extension to match NHSBSA descriptions.

| Parameter           | Value             |
|---------------------|-------------------|
| Input:              | Description       |
| Match With:         | NHSBSA Descriptions|
| Output:             | NHSBSA Description|
| Match Type:         | Approximate Match |
| Match Type Code:    | Percentage        |
| Score:              | Number (How much % match)|









# **1: Basics**

## **Input Parameters:**
Change your Sheet Name, File Name, File ID etc. here:



In [4]:
# @title Please Enter Basic Info: { run: "auto", vertical-output: true, form-width: "35%", display-mode: "form" }
option_createCSV = "No" # @param ["Yes", "No"]
# Global Variables
id_pricelistExport = ''#removed temporarily but keep it here for easy team editing
id_inputFile = ''
id_folder_templates = ''
name_inputTab = 'Input'
name_templateTab = 'Sheet1'
name_CSV = 'allTemplatesData'
name_Tab_pricelistExport = 'ProductFile'
drug_name_score_cutoff = 90
drug_description_score_cutoff = 70

# "Do you want to create CSV from Templates files? (y/n): " //keep it no unless templates have been add/deleted/edited.
option_createCSV = option_createCSV.lower()[0]
print(option_createCSV)

n


## **1.1: Imports and Installs**

In [ ]:
!pip install thefuzz

In [ ]:
import pandas as pd
import json
import csv
import re

# Import necessary libraries
from thefuzz import fuzz, process

#For Google Drive
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
from googleapiclient.http import MediaIoBaseUpload

#For Google Sheets
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

## **1.2: Modules**

In [ ]:
'''
#=============================================Introduction=========================================================
1. Required: Input Name string and Array of Standard Names
2. Takes first word from input name and finds all the standard names which contain first word from input name, with 90% accuracy. Creates a list of these matched names.
3. Uses Fuzzy match with 70% accuracy to match complete input name with short list in step 2 and finds single best match of all.
4. If no match is found in either step 1 or 2, None is returned.
#==================================================================================================================
'''
def match_input_to_standard(input_name,standard_names_list):
  try:
    # Extract the first word from input_name
    first_word_input = input_name.split()[0]

    # Match the first word with 95% accuracy
    # Create an empty shortlist
    short_list = []

    # Iterate through standard names and compare with each word in input
    for std_item in standard_names_list:
      std_words = std_item.split()
      for std_word in std_words:
        # Use fuzz.ratio for lenient matching (configurable threshold)
        if fuzz.ratio(first_word_input, std_word) >= drug_name_score_cutoff:
          short_list.append(std_item)
          break  # Exit inner loop once a match is found for the current std_item

    #print(f"short_list: {short_list}")
    match = process.extractOne(input_name, short_list, scorer=fuzz.token_sort_ratio,score_cutoff=drug_description_score_cutoff)
    if match is None:
      return (None,None)
    else:
      return match
  except Exception as e:
    print(f"Fuzzy Match: An error occurred: {e}")
    return (None,None)

In [ ]:
'''
#=============================================Introduction=========================================================
1. Fetch Sheets Data using one range input.
2. Converts each column of range to array.
3. Returns 2D array of each column as an array.
#==================================================================================================================
'''
def getSheetsData(file_id, sheet_name, data_range, major_dimension='rows'):
    try:
        data = gc.open_by_key(file_id).worksheet(sheet_name).get(data_range)
        if not data:
            return []

        if major_dimension.lower()[:3] == 'col':
            arr_data = []
            for i in range(len(data[0])):
                col_data = [row[i].lower().strip() for row in data]
                arr_data.append(col_data)
            return arr_data
        else:
            arr_data = []
            for row in data:
                arr_data.append([str(cell).lower().strip() for cell in row])
            return arr_data
    except Exception as e:
        print(f"An error occurred: {e}")
        return []



In [ ]:
#Returns Array of File IDs in a folder.

def getFileIDs_inAFolder(id_folder):
  #files = []
  fileIDs = []
  page_token = None
  while True:
    response = (drive_service.files().list(
            q=f"'{id_folder}' in parents and trashed=false and mimeType = 'application/vnd.google-apps.spreadsheet'",
            fields="nextPageToken, files(id, name)",
            pageToken=page_token,
        )
        .execute()
    )
    for file in response.get("files", []):
      # Process change
      fileIDs.append([file.get("name"),file.get('id')])
      #print(f'Found file: {file.get("name")}, {file.get("id")}')
    #files.extend(response.get("files", []))
    page_token = response.get("nextPageToken", None)
    if page_token is None:
      break
  return fileIDs

In [ ]:
#Returns Array of File IDs in a folder of a given name.
def getFileName_inAFolder(file_name,id_folder,number_of_files=1):
  fileIDs = []
  page_token = None
  while True:
    response = (drive_service.files().list(
            q=f"'{id_folder}' in parents and trashed=false and name contains '{file_name}' and mimeType = 'application/vnd.google-apps.spreadsheet'",
            fields="nextPageToken, files(id, name)",
            pageToken=page_token,
        )
        .execute()
    )
    for file in response.get("files", []):
      fileIDs.append([file.get("name"),file.get('id')])
    page_token = response.get("nextPageToken", None)
    if page_token is None:
      break
  return fileIDs[:number_of_files]

In [ ]:
def createFile(fileName,folderID,mimeType):
  file_metadata = {
    'name': fileName,
    'mimeType': mimeType,
    "parents": [folderID],
  }
  created = drive_service.files().create(body=file_metadata,fields='id').execute()
  print('File ID: {}'.format(created.get('id')))
  return created.get('id')

#createFile('allTemplates.csv',id_folder_templates)

# Function to get or create a JSON file in a specified folder
def get_or_create_file(drive_service, host_folder_id, fileName,mimeType):
    # Search for the file in the specified folder
    query = f"'{host_folder_id}' in parents and trashed=false and mimeType='{mimeType}' and name='{fileName}'"
    response = drive_service.files().list(q=query).execute()
    files = response.get('files', [])

    if files:
        # If the file exists, return its ID
        file_id = files[0]['id']
        print(f"file '{fileName}' found in the folder with ID '{host_folder_id}'. File ID: {file_id}")
        return file_id
    else:
        # If the file doesn't exist, you can create it (you mentioned you have code for this)
        print(f"file '{fileName}' not found in the folder with ID '{host_folder_id}'. Creating a new file...")
        file_id = createFile(fileName,host_folder_id,mimeType)
        return file_id

# Get or create JSON file in the specified folder
#get_or_create_file(drive_service, id_folder_templates, 'allTemplatesData','text/csv')


In [ ]:
def update_csv(file_id, new_data):
  # Convert the new data to CSV format
  csv_content = io.StringIO()
  csv_writer = csv.writer(csv_content)
  csv_writer.writerows(new_data)

  # Upload the updated content back to Google Drive, overwriting the existing file
  media_body = MediaIoBaseUpload(io.BytesIO(csv_content.getvalue().encode('utf-8')), mimetype='text/csv')
  drive_service.files().update(fileId=file_id, media_body=media_body).execute()

  print(f"Data updated successfully in the file with ID '{file_id}'.")

In [ ]:
def reArrangeCols(data, header_result,flat=False):
    header_result = [e.strip().lower() for e in header_result]
    header = [e.strip().lower() for e in data[0]]
    print(f"header Data {header}")
    reArrangedData = [header_result]

    len_rows_data = len(data)
    list_indices_result = [header.index(x) if x in header else None for x in header_result]
    print(f"Header matched with Data, list_indices_result: {list_indices_result}")

    for i in range(1,len_rows_data):
        row_to_append = []
        for j in list_indices_result:
            if j ==None:
                row_to_append.append(None)
            else:
                try:
                    row_to_append.append(data[i][j])
                except IndexError:
                    row_to_append.append(None)
        reArrangedData.append(row_to_append)
    if flat==True:
      reArrangedData = [item for sublist in reArrangedData for item in sublist]
    return reArrangedData

# Example usage
data = [['B', 'A', 'D', 'C'], [134, 454, 23],['!','£','$','%'],['-','--','---','----'],['@','@@','@@@','@@@@']]
header_result = ['A', 'B', 'E','C']
reArrangedData = reArrangeCols(data, header_result)
print(reArrangedData)


header Data ['b', 'a', 'd', 'c']
Header matched with Data, list_indices_result: [1, 0, None, 3]
[['a', 'b', 'e', 'c'], [454, 134, None, None], ['£', '!', None, '%'], ['--', '-', None, '----'], ['@@', '@', None, '@@@@']]


In [ ]:
#Read the Existing CSV File
def read_csv_file(file_id):
    try:
        request = drive_service.files().get_media(fileId=file_id)
        content = request.execute()

        # Decode the content bytes to a string
        content_str = content.decode('utf-8')

        # Create a StringIO object to simulate a file-like object
        csv_data = io.StringIO(content_str)

        # Read the CSV data into a list of dictionaries
        csv_reader = csv.DictReader(csv_data)
        rows = [row for row in csv_reader]
        return rows

    except Exception as e:
        print(f"Error reading CSV file with ID '{file_id}': {e}")
        return None


In [ ]:
def colAlphaNum(input):
    def isValidAlphabet(input):
        return bool(re.match(r'^[A-Za-z]+$', input))

    if isinstance(input, int):
        # Convert Column Number to Alphabet
        column = ''
        while input > 0:
            temp = (input - 1) % 26
            column = chr(temp + 65) + column
            input = (input - temp - 1) // 26
        return column
    elif isinstance(input, str) and isValidAlphabet(input):
        # Convert Alphabet to Column Number
        input = input.upper()
        result = 0
        base = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

        for char in input:
            charValue = base.index(char) + 1

            if charValue == 0:
                return ''  # Invalid character

            result = result * 26 + charValue

        return result

    return ''  # Return empty string for invalid input type


In [ ]:
def standardize_medicine_name(medicine_name):
  # Convert input string to lowercase
  medicine_name = medicine_name.lower().strip()

  # Replace specific terms
  replacements = {
    "tab": "tablets",
    "tabs": "tablets",
    "tablet": "tablets",

    "cap": "capsules",
    "caps": "capsules",
    "capsule": "capsules",

    "crm": "cream",
    "soln": "solution",
    "sol": "solution",
    "oint": "ointment",
    "disp": "dispersible",
    "sugar free": "s/f",
    "susp": "suspension"
  }
    # Replace last word with specified keywords
  for keyword, replacement in replacements.items():
    if medicine_name.endswith(" "+ keyword):
      medicine_name = medicine_name[:-len(keyword)] + replacement
      break
  for key, value in replacements.items():
    medicine_name = medicine_name.replace(" "+key+" ", " "+value+" ")

  # Trim and return the standardized string
  return medicine_name.strip()

# Example usage
input_string = "Paracetamol soln 500mg tab"
standardized_string = standardize_medicine_name(input_string)
print(standardized_string)

paracetamol solution 500mg tablets


# **2: Codes Matching**
Steps to match codes:
1. Get Input Data
  1. Get Supplier Name, Supplier Codes
  2. Get PIP Code, Supplier Description
2. Find a Supplier Template, get data
3. Get All Templates CSV Data
  1. Create CSV from Templates
  2. Read existing CSV data
4. Get PricelistExport Database
5. Loop on Input Lines:
  1. Find `Supplier Code` in Supplier Template
  2. If blank, Find `PIP Code` in CSV Templates (which includes supplier tempate as well)
  3. If blank, Find `PIP Code` in PricelistExport Database
  4. If blank, Find `Supplier Description` in CSV Templates
  5. If blank, Find `Supplier Description` in PricelistExport Database
  6. If blank, Fuzzy Match `Supplier Description` with PricelistExport Database


## **2.1: Reading Input data from Input file**

In [ ]:
input_data = getSheetsData(id_inputFile,name_inputTab,'A1:Z',major_dimension='rows')
header_input_data = input_data[0]
#input_names = reArrangeCols(input_data,['Supplier Description'],flat=True)
#print(input_names[:5])

index_pip_code = header_input_data.index('pip code')
index_supplier_description = header_input_data.index('supplier description')
index_supplier_code = header_input_data.index('supplier code')
index_internal_code = header_input_data.index('internal code')

col_InternalCode = colAlphaNum(index_internal_code +1)
col_Score = colAlphaNum(header_input_data.index('score')+1)
col_MatchType = colAlphaNum(header_input_data.index('match type')+1)

df_input_data = pd.DataFrame(input_data,columns=header_input_data)
if 'supplier:' in header_input_data:
  supplier_Name = header_input_data[header_input_data.index('supplier:') + 1]
else:
  supplier_Name = None
print(f"supplier_Name: {supplier_Name}")

supplier_Name: dataplast


##**2.2: Find A Supplier Template and Get Data**

In [ ]:
template_Supplier = getFileName_inAFolder(supplier_Name,id_folder_templates,number_of_files=1)
if template_Supplier:
  template_Supplier = template_Supplier[0]
  print(f"template_Supplier:{template_Supplier}")

  template_Supplier_data = getSheetsData(template_Supplier[1],name_templateTab,'A1:Z')
  print(template_Supplier_data[:5])

  df_supplier_template = pd.DataFrame(template_Supplier_data[1:],columns=template_Supplier_data[0])
  print(df_supplier_template.head(5))

  df_supplier_template.set_index('supplier code', inplace=True)
else:
  df_supplier_template = pd.DataFrame()

template_Supplier:['Dataplast', '1dKfI63MlL3boKOTRVrcD92sHsQqBA47kydfv4nEo-Qk']
[['internal code', 'supplier code', 'product', 'price1', 'supplier1', 'price2', 'supplier2', 'usage1', 'dtprice', 'cdcat', 'fridge', 'verify'], ['', '', '1 a1 calcium 1.25g chewable tablets s/f 100', '£3.46', 'aah', '£3.48', 'unichem', '1', '0'], ['', '', '1 a1 calcium d3 400iu/1.25g chewable tablets 100', '£4.33', 'aah', '£4.33', 'unichem', '1', '0'], ['', '', '1 a1 calcium d3 400iu/1.25g chewable tablets 60', '£2.60', 'unichem', '£3.03', 'aah', '0', '0'], ['', '', '1 a1 calcium d3 400iu/1.5g chewable tablets 112', '£4.25', 'aah', '£4.28', 'unichem', '0', '0']]
  internal code supplier code  \
0                               
1                               
2                               
3                               
4                               

                                            product price1 supplier1 price2  \
0       1 a1 calcium 1.25g chewable tablets s/f 100  £3.46       aah  £3.

## **2.3: Get All Templates CSV Data**

### **2.3.1: Create CSV from Templates**

In [ ]:
completedTemplates = []
allData_Templates = [['internal code','pip code','supplier description']]
id_allDataCSV = get_or_create_file(drive_service, id_folder_templates, name_CSV,'text/csv')

file 'allTemplatesData' found in the folder with ID '105WtQwhjXQ2pBAg9f0vMJR8BYeM6gqXO'. File ID: 1ni4EtmdGc1a5UtG7LdSrNjwB-p5yu4qY


In [ ]:
# Check if the user wants to process the step
if option_createCSV.lower() == 'y':
  # Process the step
  print("Processing Templates to create CSV")

  templates = getFileIDs_inAFolder(id_folder_templates)
  print(templates)
  header = allData_Templates[0]
  for index,template in enumerate(templates):
    #try:
      print(f"                                          ")
      print(f"-----------------------------------------")
      print(f"index:{index}, {template}")
      #if index==5:
      #  break
      if template[1] in completedTemplates:
        print('Skip')
        continue
      else:
        #print(template)
        sheetData = getSheetsData(template[1],name_templateTab,'A1:Z','rows')
        if not sheetData:
          continue
        sheetData = reArrangeCols(sheetData,header)
        print(sheetData[:5])
        allData_Templates.extend(sheetData[1:])
        update_csv(id_allDataCSV, allData_Templates)
        completedTemplates.append(template[1])
    #except Exception as e:
        #print(f"An error occurred: {e}")
else:
  # Skip the step
  print("Skip the creating CSV step.")

Do you want to create CSV from Templates files? (y/n): y
Processing Templates to create CSV
[['Drugs R Us', '1tEqitYNlkZNDUAK03dWSD35VXdj0t6bx1HeNi3kzTsU'], ['Dataplast', '1dKfI63MlL3boKOTRVrcD92sHsQqBA47kydfv4nEo-Qk'], ['Day Lewis', '1MQ_EUusqS5Nmw3fJk8JkqWUG84d2Fvxt5xsluwfVP4I'], ['Sample Supplier', '11UEIroUdzDVSxtE8K_FvEh5_na2h8n9fQEUPkkEG8_Y'], ['A1', '1xbSmMRaP_hiCxd0T2llUEXuMXP5q1IRoO-p8v3vEwkw'], ['Bestway', '16SUgJndTgLvK33gdqUvRMfZ1yNUE3kcvDaZJCZq37gc'], ['Astute', '1oLoMHiaaGRJs0qjaT1I1KG7VucIsSOvXJOhVrlyEE18'], ['Medihealth', '1DueaAz6YpR_ZO8BX2tHnjanIbqaMTjmqAr9KiIX1JKs'], ['Nupharm', '1Ucs65ufoqY7C0PCS4ramu0S4eF-T-0kCA-LzyE4O_HA'], ['AAH Generics', '1IvAVt_2i_n-JKK0o6Od32P1NazocCVjpLG9SDFK2kDI'], ['Alliance Netgen', '10CUJfUvTvO6TCP2oC12YVJZ-itIM-pfKD_tQ0taiRpk'], ['AAH Net Price', '1KE2-7Pqc3BwtInn225cYSqWJaHfNG8ATI7p89STprdo'], ['Smartways', '1zdkO3IwxGIIbC4jLf84GjB2Y_wquwQP4HPrRTIUjCjA'], ['Colorama', '1Wc2llbgEmxUMoUsPXYAS8U762LwULd6qpewbyHzV1mU'], ['Pura', '1kP7QG5Lo

In [ ]:
print(len(completedTemplates))
print('A1' in completedTemplates)

0
False


### **2.3.2: Read existing CSV data**

In [ ]:
#Read Templates CSV
templates_data = read_csv_file(id_allDataCSV)
print(templates_data[:5])
df_csv = pd.DataFrame(templates_data[1:],columns=templates_data[0])
df_csv.set_index('supplier description',inplace=True)
print(df_csv.head(5))

[{'internal code': 'abas100kpen5', 'pip code': '4025029', 'supplier description': 'abasaglar kwik pen 100u/ml 5x3ml'}, {'internal code': 'abil10mgtab28', 'pip code': '', 'supplier description': 'abilify 10mg tabs x 28'}, {'internal code': 'abil15mgtab28', 'pip code': '3051471', 'supplier description': 'abilify 15mg tabs x 28'}, {'internal code': 'abil5mgtab28', 'pip code': '3126091', 'supplier description': 'abilify 5mg tabs x 28'}, {'internal code': 'abil1mgmora150', 'pip code': '3224722', 'supplier description': 'abilify oral solution 1mg/ml x 150ml'}]
                                       internal code pip code
supplier description                                         
abilify 10mg tabs x 28                 abil10mgtab28         
abilify 15mg tabs x 28                 abil15mgtab28  3051471
abilify 5mg tabs x 28                   abil5mgtab28  3126091
abilify oral solution 1mg/ml x 150ml  abil1mgmora150  3224722
accu-chek aviva test strips x 50      accucheckavi50  3171253


## **2.4: Get PricelistExport Database**

In [ ]:
pricelistExport_data = getSheetsData(id_pricelistExport,name_Tab_pricelistExport,'A1:Z')
internal_codes = reArrangeCols(pricelistExport_data,['Internal Code'],flat=True)
standard_names_list = reArrangeCols(pricelistExport_data,['Item Name'],flat=True)
print(internal_codes[:5])
print(standard_names_list[:5])
df_pricelistExport = pd.DataFrame(pricelistExport_data[1:],columns=pricelistExport_data[0])
df_pricelistExport.set_index('item name',inplace=True)
print(df_pricelistExport.head(5))
print(f"Missing Values: {df_pricelistExport.index.isnull().sum()}")

header Data ['internal code', 'item name', 'pip code']
Header matched with Data, list_indices_result: [0]
header Data ['internal code', 'item name', 'pip code']
Header matched with Data, list_indices_result: [1]
['internal code', 'coqe100mgcap30', 'coqe30mgcap30', 'mgci4166435', 'mgox4158259']
['item name', '10-q co-enzyme q10 100mg capsules 30', '10-q co-enzyme q10 30mg capsules 30', '100 m-trate magnesium citrate 100mg caps 30', '100mag magnesium oxide 100mg capsules 30 caps']
                                                internal code pip code
item name                                                             
10-q co-enzyme q10 100mg capsules 30           coqe100mgcap30     None
10-q co-enzyme q10 30mg capsules 30             coqe30mgcap30     None
100 m-trate magnesium citrate 100mg caps 30       mgci4166435  4166435
100mag magnesium oxide 100mg capsules 30 caps     mgox4158259  4158259
160mag magnesium oxide 160mg capsules 30 caps     mgox4158234  4158234
Missing Values: 0


## **2.5: Loop on Input Lines:**

In [ ]:
# Match the Internal Codes
countUnMatch = 0
totalCount = len(input_data[1:])
list_internal_codes = []
list_scores = []
list_match_type = []
step = totalCount // 10
progress = 0

for index, input_row in df_input_data.iloc[1:].iterrows():
  if index >0 and index % step == 0:
    progress += 10
    print(f"Progress: {progress}%")
  internal_code = input_row['internal code']
  score = None
  match_type = None

  input_code = input_row['supplier code']
  input_pip = input_row['pip code']
  input_name = input_row['supplier description']

  ### 2.5.1: Find Supplier Code in Supplier Template
  if not internal_code and input_code and not df_supplier_template.empty and input_code in df_supplier_template.index:
    internal_code = df_supplier_template.loc[input_code, 'internal code']
    if type(internal_code) != type('string'):
      internal_code = internal_code.iloc[0]
    match_type = 'Supplier Code'

  ### 2.5.2: Find PIP Code in CSV Templates
  if not internal_code and input_pip and input_pip in df_csv['pip code'].values:
    internal_code = df_csv.loc[df_csv['pip code'] == input_pip, 'internal code'].iloc[0]
    match_type = 'PIP - Templates'

  ### 2.5.3: Find PIP Code in PricelistExport Database
  if not internal_code and input_pip and input_pip in df_pricelistExport['pip code'].values:
    internal_code = df_pricelistExport.loc[df_pricelistExport['pip code'] == input_pip, 'internal code'].iloc[0]
    match_type = 'PIP - PricelistExport'

  ### 2.5.4: Find Supplier Description in CSV Templates
  if not internal_code and input_name and input_name in df_csv.index:
    internal_code = df_csv.loc[input_name,'internal code']
    if type(internal_code) != type('string'):
      internal_code = internal_code.iloc[0]
    match_type = 'Item - Templates'

  ### 2.5.5: Find Supplier Description in PricelistExport Database
  if not internal_code and input_name and input_name in df_pricelistExport.index:
    internal_code = df_pricelistExport.loc[input_name,'internal code']
    if type(internal_code) != type('string'):
      internal_code = internal_code.iloc[0]
    match_type = 'Item - PricelistExport'

  ### 2.5.6: Fuzzy Match Supplier Description with PricelistExport Database
  if not internal_code and input_name:
    matched_standard_name,score = match_input_to_standard(standardize_medicine_name(input_name),standard_names_list)
    #print(f"input:{input_name}, result: {matched_standard_name}")
    if matched_standard_name and matched_standard_name in df_pricelistExport.index:
      internal_code = df_pricelistExport.loc[matched_standard_name,'internal code']
      if type(internal_code) != type('string'):
        internal_code = internal_code.iloc[0]
      match_type = 'Percentage'

  if not internal_code:
    countUnMatch+=1
    match_type = None
  list_internal_codes.append([str(internal_code).upper() if internal_code else None])
  list_scores.append([score])
  list_match_type.append([match_type])


print(f"Success Ratio: {((totalCount-countUnMatch)*100)/totalCount} -Count UnMatched: {countUnMatch}/{totalCount}")

gc.open_by_key(id_inputFile).worksheet('Input').batch_update(
    [{'range':f'{col_InternalCode}2:{col_InternalCode}','values': list_internal_codes},
     {'range':f'{col_Score}2:{col_Score}','values':list_scores},
     {'range':f'{col_MatchType}2:{col_MatchType}','values':list_match_type}
     ]
    )


Progress: 10%
Progress: 20%
Progress: 30%
Progress: 40%
Progress: 50%
Progress: 60%
Progress: 70%
Progress: 80%
Progress: 90%
Progress: 100%
Success Ratio: 32.54959359720968 -Count UnMatched: 21659/32111


{'spreadsheetId': '1ZW7Go5CwiQDtfLrlPRshV16za-wOzgHSjypiPTB8fUk',
 'totalUpdatedRows': 10452,
 'totalUpdatedColumns': 3,
 'totalUpdatedCells': 20660,
 'totalUpdatedSheets': 1,
 'responses': [{'spreadsheetId': '1ZW7Go5CwiQDtfLrlPRshV16za-wOzgHSjypiPTB8fUk',
   'updatedRange': 'Input!D2:D32112',
   'updatedRows': 10452,
   'updatedColumns': 1,
   'updatedCells': 10452},
  {'spreadsheetId': '1ZW7Go5CwiQDtfLrlPRshV16za-wOzgHSjypiPTB8fUk',
   'updatedRange': 'Input!E2:E32112',
   'updatedRows': 4488,
   'updatedColumns': 1,
   'updatedCells': 4488},
  {'spreadsheetId': '1ZW7Go5CwiQDtfLrlPRshV16za-wOzgHSjypiPTB8fUk',
   'updatedRange': 'Input!F2:F32112',
   'updatedRows': 5720,
   'updatedColumns': 1,
   'updatedCells': 5720}]}

# **3: NHSBSA Description Matches**